# Company Fundamentals Analysis

In [1]:
import pandas as pd
import yfinance as yf
import yahooquery as yq
import math

In [2]:
hold = pd.read_csv("holdtyped.csv", index_col=0)

In [3]:
holdings = hold.loc[hold["Type"] == "EQ"]

In [4]:
holdings.columns = pd.MultiIndex.from_tuples([(x,"") for x in holdings.columns], names=["Header", "Sub-Header"])

In [5]:
fin = pd.DataFrame(yq.Ticker("TCS.NS").income_statement('a',False))["asOfDate"]

In [6]:
cols = [ x for x in fin ]

In [7]:
holdings = holdings.set_index("Symbol")

In [8]:
sym = holdings.index

In [9]:
financials = pd.DataFrame(yq.Ticker(list(sym)).income_statement('a',False))
financials = financials.swapaxes("columns","index")

In [10]:
financials.columns = pd.MultiIndex.from_arrays([financials.columns, financials.loc["asOfDate"].values])

In [11]:
for i in holdings.index:
    basicEPS = financials.loc["BasicEPS", i]
    for j in cols:
        try:
            if math.isnan(basicEPS[j]):
                raise Exception("NA")
            holdings.loc[i, ("Basic EPS",j)] = basicEPS[j]
        except:
            holdings.loc[i, ("Basic EPS",j)] = None

In [12]:
holdings = holdings.reset_index()
holdings.to_csv("eqholdweps.csv")